In [1]:
import psycopg2

In [2]:
import csv

In [3]:
import pandas as pd

In [4]:
import numpy as np

In [5]:
from tabulate import tabulate

In [6]:
conn = psycopg2.connect("host=localhost port=5432 dbname=events user=postgres password=seqret")
cursor = conn.cursor()

### Preparing the table 

In [6]:
df = pd.read_csv('athletes_events.csv')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271116 entries, 0 to 271115
Data columns (total 15 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   ID      271116 non-null  int64  
 1   Name    271116 non-null  object 
 2   Sex     271116 non-null  object 
 3   Age     261642 non-null  float64
 4   Height  210945 non-null  float64
 5   Weight  208241 non-null  float64
 6   Team    271116 non-null  object 
 7   NOC     271116 non-null  object 
 8   Games   271116 non-null  object 
 9   Year    271116 non-null  int64  
 10  Season  271116 non-null  object 
 11  City    271116 non-null  object 
 12  Sport   271116 non-null  object 
 13  Event   271116 non-null  object 
 14  Medal   39783 non-null   object 
dtypes: float64(3), int64(2), object(10)
memory usage: 31.0+ MB


In [8]:
df = df.fillna(0)
df['Height'] = df['Height'].astype("Int64")
df['Age'] = df['Age'].astype("Int64")

In [9]:
df.to_csv('athletes_sql.csv')

In [10]:
query = """
CREATE TABLE athletes_events (
    id SERIAL PRIMARY KEY,
    athlete_id INTEGER,
    name VARCHAR,
    sex VARCHAR,
    age INTEGER,
    height INTEGER,
    weight REAL,
    team VARCHAR,
    noc VARCHAR,
    games VARCHAR,
    year INTEGER,
    season VARCHAR,
    city VARCHAR,
    sport VARCHAR,
    event VARCHAR,
    medal VARCHAR
)
"""

In [11]:
cursor.execute(query)
conn.commit()

In [12]:
with open('athletes_sql.csv', 'r') as file:
    reader = csv.reader(file)
    # Skip the header row
    next(reader)
    for ID, row in enumerate(reader):
        cursor.execute(
            "INSERT INTO athletes_events VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)",
            row
        )
conn.commit()

In [7]:
def fetch_all(cursor):
    colnames = [desc[0] for desc in cursor.description]
    records = cursor.fetchall()
    return [{colname: value for colname, value in zip(colnames, record)} for record in records]

In [8]:
cursor.execute(
    """
    SELECT * FROM athletes_events Limit 5
    
    """
)
print(tabulate(fetch_all(cursor), "keys", "psql"))

+------+--------------+--------------------------+-------+-------+----------+----------+----------------+-------+-------------+--------+----------+-----------+---------------+----------------------------------+---------+
|   id |   athlete_id | name                     | sex   |   age |   height |   weight | team           | noc   | games       |   year | season   | city      | sport         | event                            | medal   |
|------+--------------+--------------------------+-------+-------+----------+----------+----------------+-------+-------------+--------+----------+-----------+---------------+----------------------------------+---------|
|    0 |            1 | A Dijiang                | M     |    24 |      180 |       80 | China          | CHN   | 1992 Summer |   1992 | Summer   | Barcelona | Basketball    | Basketball Men's Basketball      | 0       |
|    1 |            2 | A Lamusi                 | M     |    23 |      170 |       60 | China          | CHN   | 20

### 1. How old were the youngest male and female participants of the 1996 Olympics?

In [10]:
#for females
cursor.execute(
    """
    SELECT MIN(Age) AS "min_f" FROM athletes_events WHERE Sex = 'F' and Age > 0 and Year = 1996
    
    """
)
print(tabulate(fetch_all(cursor), "keys", "psql"))

+---------+
|   min_f |
|---------|
|      12 |
+---------+


In [11]:
#for males
cursor.execute(
    """
    SELECT MIN(Age) AS "min_m" FROM athletes_events WHERE Sex = 'M' and Age > 0 and Year = 1996
    
    """
)
print(tabulate(fetch_all(cursor), "keys", "psql"))

+---------+
|   min_m |
|---------|
|      14 |
+---------+


### 2. What was the percentage of male gymnasts among all the male participants of the 2000 Olympics? Round the answer to the first decimal.

In [9]:
cursor.execute(
    '''
    SELECT ROUND((COUNT(DISTINCT(athlete_id)) / (SELECT COUNT(DISTINCT(athlete_id))
    FROM athletes_events WHERE sex = 'M' and year = 2000)::decimal) * 100, 1) AS "percentage"
        FROM athletes_events
        WHERE sex = 'M' and year = 2000 and sport = 'Gymnastics'
    '''
)
print(tabulate(fetch_all(cursor), "keys", "psql"))

+--------------+
|   percentage |
|--------------|
|          1.5 |
+--------------+


### 3. What are the mean and standard deviation of height for female basketball players participated in the 2000 Olympics? Round the answer to the first decimal.

In [8]:
cursor.execute(
    '''
    SELECT ROUND((AVG(temp.height)::numeric), 1) AS "AVG_height",
    ROUND((STDDEV(temp.height)::numeric), 1) AS "STD_height"
    FROM (SELECT DISTINCT athlete_id, height FROM athletes_events
    WHERE sex = 'F' and year = 2000 and height > 0 and sport = 'Basketball') AS temp
    '''
)

print(tabulate(fetch_all(cursor), "keys", "psql"))

+--------------+--------------+
|   AVG_height |   STD_height |
|--------------+--------------|
|        182.4 |          9.1 |
+--------------+--------------+


### 4. Find a sportsperson participated in the 2002 Olympics, with the highest weight among other participants of the same Olympics. What sport did he or she do?

In [12]:
cursor.execute(
    """
    SELECT sport FROM athletes_events
    WHERE weight=(SELECT MAX(weight) FROM athletes_events WHERE year = 2002) and year = 2002
    """
)
print(tabulate(fetch_all(cursor), "keys", "psql"))

+-----------+
| sport     |
|-----------|
| Bobsleigh |
+-----------+


### 5. How many times did Pawe Abratkiewicz participate in the Olympics held in different years?

In [13]:
cursor.execute(
    """
    SELECT COUNT(DISTINCT temp.games)
    FROM (SELECT DISTINCT name, games FROM athletes_events
    WHERE name = 'Pawe Abratkiewicz') AS temp
    """
)
print(tabulate(fetch_all(cursor), "keys", "psql"))

+---------+
|   count |
|---------|
|       3 |
+---------+


### 6. How many silver medals in tennis did Australia win at the 2000 Olympics?

In [14]:
cursor.execute(
    """
    SELECT COUNT (*) FROM athletes_events
    WHERE team = 'Australia' and year = 2000 and sport = 'Tennis' and medal ='Silver'
    """
)
print(tabulate(fetch_all(cursor), "keys", "psql"))

+---------+
|   count |
|---------|
|       2 |
+---------+


### 7. Is it true that Switzerland won fewer medals than Serbia at the 2016 Olympics?

In [10]:
cursor.execute(
    """
    SELECT(SELECT COUNT(*)
    FROM athletes_events
    WHERE year = 2016 and team = 'Serbia' and medal != '0') >
    (SELECT COUNT(*)
    FROM athletes_events
    WHERE year = 2016 and team = 'Switzerland' and medal != '0') AS answer
    """
)
print(tabulate(fetch_all(cursor), "keys", "psql"))

+----------+
| answer   |
|----------|
| True     |
+----------+


### 8. What age category did the fewest and the most participants of the 2014 Olympics belong to?

In [15]:
cursor.execute(
    """
    SELECT
        CASE
            WHEN 15 <= age AND 25 > age THEN 15
            WHEN 25 <= age AND 35 > age THEN 25
            WHEN 35 <= age AND 45 > age THEN 35
            WHEN 45 <= age AND 55 >= age THEN 45
        END AS age_category,
    COUNT (DISTINCT athlete_id)
    FROM athletes_events
    WHERE year = 2014
    GROUP BY age_category
"""
)
print(tabulate(fetch_all(cursor), "keys", "psql"))

+----------------+---------+
|   age_category |   count |
|----------------+---------|
|             15 |    1193 |
|             25 |    1397 |
|             35 |     150 |
|             45 |       5 |
+----------------+---------+


[45-55] and [25-35) correspondingly

### 9. Is it true that there were Summer Olympics held in Lake Placid? Is it true that there were Winter Olympics held in Sankt Moritz?

In [8]:
cursor.execute(
    """
    SELECT (SELECT COUNT(*) > 0
    FROM athletes_events
    WHERE city = 'Lake Placid' and season = 'Summer') as Lake_Placid,
    (SELECT COUNT(*) > 0 FROM athletes_events
    WHERE city = 'Sankt Moritz' and season = 'Winter') as Sankt_Moritz
    """
)
print(tabulate(fetch_all(cursor), "keys", "psql"))

+---------------+----------------+
| lake_placid   | sankt_moritz   |
|---------------+----------------|
| False         | True           |
+---------------+----------------+


### 10. What is the absolute difference between the number of unique sports at the 1996 Olympics and 2016 Olympics?

In [10]:
cursor.execute(
    """
    SELECT ABS(
    (SELECT COUNT (DISTINCT sport) FROM athletes_events where year = 1996)-
    (SELECT COUNT (DISTINCT sport) FROM athletes_events where year = 2016)) AS difference
    """
)
print(tabulate(fetch_all(cursor), "keys", "psql"))

+--------------+
|   difference |
|--------------|
|            3 |
+--------------+
